For working with the model, we first import some packages and the modelclass:

In [4]:
#os.chdir('C:\\Users\\mpher\\Documents\\Uni\\Master\\02_Exchange\\01_Academics\\Energy Economics of the Green Transition\\0_Final Project\EnergyEconGroupWork')

In [5]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/paul/EnergyEcon/EnergyEconGroupWork


In [6]:
%run ./stdPackages.ipynb
from pyDbs import *

os.chdir(os.path.join(d['curr'], 'py'))

from mBasicPH_storage import *
os.chdir(d['curr'])

Next we load the database:

In [7]:
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'GeneratorsVariables','StorageVariables','TransmissionLines'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps', 'MarketMaps'],
        'variable2D': ['HourlyVariation']}

path_to_data = os.path.join(d['curr'], 'Data', 'mBasicPH_storage_Data.xlsx')

db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

Convert variables to floats:

In [8]:
[db.__setitem__(var,db[var].astype(float).sort_index()) for var in db.getTypes('variable').keys()];

We need to read in heat areas manually:

In [9]:
db['g_H'] = db['g_E2g_H'].get_level_values('g_H')

Collect all areas in one:

In [10]:
db['g'] = db['g_E'].copy()

Finally, we will assume that linelosses and trade are zero such that the marginal willingness to pay and marginal generation costs of "transmission lines" does not have to be corrected. 

We then initialize and solve the model:

In [11]:
m = mSimple(db)
m()

/Users/paul/anaconda3/envs/EE2023/lib/python3.11/site-packages/pyDbs/base.py:289: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  return x.sort_index().add(pd.Series(0, index = y).sort_index(),fill_value=fill_value) if (set(x_dom)-set(y_dom)) else pd.Series(0, index = y).sort_index().add(x.sort_index(),fill_value=fill_value)
/Users/paul/anaconda3/envs/EE2023/lib/python3.11/site-packages/pyDbs/base.py:289: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  return x.sort_index().add(pd.Series(0, index = y).sort_index(),fill_value=fill_value) if (set(x_dom)-set(y_dom)) else pd.Series(0, index = y).sort_index().add(x.sort_index(),fill_value=fill_value)
/Users/paul/anaconda3/envs/EE2023/lib/python3.11/site-packages/pyDbs/base.py:289: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  return x.sort_index().add(pd.Series(0, index = y).s

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


In [48]:
# Example: Assuming 'Welfare', 'FuelConsumption', 'Emissions', etc., are columns in your DataFrame
result_columns = ['Welfare', 'FuelConsumption', 'Emissions', 'marginalSystemCosts_E', 'marginalSystemCosts_H', 'marginalEconomicValue', 'meanConsumerPrice_E', 'meanConsumerPrice_H']
# Extract relevant columns from self.db and create a DataFrame
result_df = pd.DataFrame({column: m.db[column] for column in result_columns})

# Store the Solutions in a Pandas DataFrame
#m.db = {key: [value] if not isinstance(value, (list, tuple)) else value for key, value in m.db.items()}
#result_df = pd.DataFrame.from_dict(m.db, orient='columns')

# Export the DataFrame to Excel
result_df.to_excel(os.path.join(d['curr'], 'solutions.xlsx'), index=False, engine='openpyxl')

/var/folders/yp/j0c37_w920sg_vxm7smcssjw0000gn/T/ipykernel_18294/1552974739.py:4: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result_df = pd.DataFrame({column: m.db[column] for column in result_columns})
/var/folders/yp/j0c37_w920sg_vxm7smcssjw0000gn/T/ipykernel_18294/1552974739.py:4: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result_df = pd.DataFrame({column: m.db[column] for column in result_columns})
/var/folders/yp/j0c37_w920sg_vxm7smcssjw0000gn/T/ipykernel_18294/1552974739.py:4: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result_df = pd.DataFrame({column: m.db[column] for column in result_columns})


What to plot?

- CO2 Emissions
- Usage of Storage
- Marginal Economic value of storage
- seasonal storage?